# Import ploonetide and other libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ploonetide import TidalSimulation
from ploonetide.utils import colorline
from ploonetide.utils.functions import mean2axis, find_moon_fate
from ploonetide.utils.constants import GYEAR, KYEAR, DAY, MSUN, AU

-------
## Create TidalSimulation object for planet-moon system. The list of the most important parameters is printed out.

In [2]:
simulation = TidalSimulation(
    system='planet-moon',
    star_mass=0.8,  # Solar masses
    star_radius=1.0,  # Solar radii
    star_rotperiod=10,  # days
    planet_mass=2,  # Jupiter masses
    planet_radius=1.3,  # Jupiter radii
    planet_orbperiod=20,  # days
    planet_rotperiod=0.6,  # days
    moon_semimaxis=10,  # Roche radii
    moon_eccentricty=0.0,
    planet_size_evolution=False,
    planet_internal_evolution=False,
    planet_core_dissipation=False,
)

       _                        _   _     _      
 _ __ | | ___   ___  _ __   ___| |_(_) __| | ___ 
| '_ \| |/ _ \ / _ \| '_ \ / _ \ __| |/ _` |/ _ \
| |_) | | (_) | (_) | | | |  __/ |_| | (_| |  __/
| .__/|_|\___/ \___/|_| |_|\___|\__|_|\__,_|\___|
|_|                                              


Stellar mass: 0.8 solMass
 Planet orbital period: 20.0 d
 Planetary mass: 2.0 jupiterMass
 Planetary radius: 1.3 jupiterRad
 Moon orbital period: 10.0 a_roche (2.6 d days)



----
## After building the TidalSimulation class instance, and before running the simulation, you still can modify any parameter in `dir(simulation)`:

In [13]:
# print(dir(simulation))  # Uncomment this line to print all the simulation attributes
simulation.star_mass = 1.0  # Solar masses
simulation.planet_mass = 2.2  # Jupiter masses

In [15]:
# Now let's print the list of all the parameters
# to be used (everything is in SI units)
simulation.sim_parameters

{'Ms': 1.988409870698051e+30,
 'Rs': 695700000.0,
 'Ls': 6.463570700718025e+25,
 'coeff_star': 0.5,
 'star_alpha': 0.25,
 'star_beta': 0.25,
 'os_saturation': 4.3421e-05,
 'star_age': 1.57788e+17,
 'coeff_planet': 0.26401,
 'Mp': 4.1758741141393115e+27,
 'Rp': 92939600.0,
 'planet_alpha': 0.126,
 'planet_beta': 0.02,
 'rigidity': 44600000000.0,
 'E_act': 300000.0,
 'B': 25.0,
 'Ts': 1600.0,
 'Tb': 1800.0,
 'Tl': 2000.0,
 'Cp': 1260.0,
 'ktherm': 2.0,
 'Rac': 1100.0,
 'a2': 1.0,
 'alpha_exp': 0.0001,
 'densm': 5495.021865555213,
 'Mm': 5.972167867791379e+24,
 'Rm': 6378100.0,
 'melt_fr': 0.5,
 'sun_omega': 2.67e-06,
 'sun_mass_loss_rate': 882124692.301465,
 'nm_ini': 2.9088649392158994e-05,
 'np_ini': 3.636102608321527e-06,
 'op_ini': 0.00012120342027738399,
 'Tm_ini': 429.61604317790363,
 'Em_ini': 0.0,
 'em_ini': 0.0,
 'args': {'star_internal_evolution': False,
  'star_k2q': 2.48051072700778e-07,
  'planet_internal_evolution': False,
  'planet_k2q': 6.078241478466431e-05,
  'planet_si

-----
### Choose the total integration time and time-step

In [18]:
integration_time = 1 * simulation.star_lifespan.value
time_step = 10 * KYEAR

### Choose the integrator and run the simulation

In [ ]:
simulation.set_integration_method('rk4')
simulation.run(integration_time, time_step)

Progress:  25%|████████████████████████████████▍                                                                                                 | 249906/1000000 steps | 02:33<06:29

----
### Get the times and solutions

In [6]:
times, solutions = simulation.history
nms = solutions[:, 0]
ops = solutions[:, 1]
nps = solutions[:, 2]
Tms = solutions[:, 3]
Ems = solutions[:, 4]
if simulation.moon_eccentricty != 0.0:
    ems = solutions[:, 5]

### Plot the solutions using matplotlib and a few helper functions from ploonetide 

In [ ]:
# COMPUTE SEMIMAJOR AXIS
ams = mean2axis(nms, simulation.planet_mass.to('kg').value, simulation.moon_mass.to('kg').value)
aps = mean2axis(nps, simulation.star_mass.to('kg').value, simulation.planet_mass.to('kg').value)

# ************************************************************
# FINDING ROUND-TRIP TIMES OR RUNAWAY TIMES
# ************************************************************
fate = find_moon_fate(times, ams, simulation.moon_roche_radius.value, simulation.planet_orbperiod.to('s').value,
                      simulation.planet_mass.to('kg').value, simulation.star_mass.to('kg').value)

labels = {'P': r'$\mathrm{Log_{10}}(P_\mathrm{orb})\mathrm{[d]}$',
          'Ms': r'$M_\bigstar[\mathrm{M_\odot}]$', 'Mp': r'$M_\mathrm{p}[\mathrm{M_{jup}}]$',
          't': r'$\mathrm{Log_{10}}(\mathrm{Time})\mathrm{[Gyr]}$'}

labelsize = 7.
markersize = 7.
ticksize = 6.

x = times / GYEAR
y = ams / simulation.moon_roche_radius.value
z = Tms

fig = plt.figure(figsize=(5, 3.5))
ax = fig.add_subplot(1, 1, 1)
lc = colorline(x, y, z, cmap='jet')
cbar = fig.colorbar(lc, orientation='vertical', aspect=17, format="%.2f", pad=0.04)
cbar.set_label(label='Temperature [K]', size=7)
cbar.set_ticks(np.linspace(np.nanmin(z), np.nanmax(z), 9))
cbar.ax.tick_params(labelsize=ticksize)
# ax.axhline(am_roche / roche_lims, c="k", ls="--", lw=0.9, zorder=0.0, label="Roche limit")

ax.set_xlim(x.min(), x[fate.index] + 0.2)
ax.set_ylim(1.0, np.nanmax(y[np.isfinite(y)]) + 1)
ax.tick_params(axis='both', direction='in', labelsize=ticksize)

ax.set_xlabel('Time [Gyr]', fontsize=labelsize)
ax.set_ylabel(r'Moon semi-major axis [$a_\mathrm{Roche}$]', fontsize=labelsize)
# ax.legend(loc='upper right', fontsize=labelsize)
fig.savefig('migration.png', dpi=300, facecolor='w')
